In [1]:
# link to tutorial: https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html


In [3]:
# pytorch has two primitives to work with data: torch.utils.data.DataLoader 
# and torch.utils.data.Dataset
# Dataset stores the samples and their corresponding labels
# DataLoader wraps an iterable around the dataset.

In [1]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# torchvision.datasets module contains Dataset objects for vision datasets

In [6]:
# for this tutorial we are using FashionMNIST dataset

In [7]:
# every Torchvision dataset includes two arguments: transform and target_transform to modify their samples and labels respectively

In [2]:
# download the training data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# download test data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
# pass the Dataset as an argument to DataLoader.
# this wraps an iterable over our dataset and supports automatic batching,
# supports sampling, shuffling, and multiprocess data loading
# here a batch size of 64 is defined. 
# each element in the dataloder iterable will return a batch of 64 features 
# and labels.

In [3]:
batch_size = 64
# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of your: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of your: torch.Size([64]) torch.int64


In [5]:
# creating models to define a neural network in pytorch, create a class
# that inherits from nn.Module. we define the layers of the network
# in the __init__ function and specify how data will pass through the
# network in the forward function. to accelerate operations in the 
# neural network, we move it to the accelerator such as CUDA, MPS, MTIA
# or XPU. if the current accelerator is available, use it otherwise the CPU


In [5]:
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
#print(f"Using {device} device")

# define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# optimizing the model parameters 
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
# in a single training loop, the model makes predictions on the training
# dataset (fed to it in batches) and backpropagates the prediction error
# to adjust the models parameters

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        #X, y = X.to(device), y.to(device)
        
        # compute the prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropogation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            #X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
# the training process is conducted oer several iterations (epochs).
# during each epoch the model learns parameters to make better predictions
# model accruacy and loss at each epoch is printed
# goal is to see the accuracy increase and loss decrease with each epoch


In [15]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------
loss: 2.301434 [   64/60000]
loss: 2.290249 [ 6464/60000]
loss: 2.269104 [12864/60000]
loss: 2.261701 [19264/60000]
loss: 2.260978 [25664/60000]
loss: 2.228773 [32064/60000]
loss: 2.241886 [38464/60000]
loss: 2.210969 [44864/60000]
loss: 2.199278 [51264/60000]
loss: 2.177075 [57664/60000]
Test Error: 
 Accuracy: 39.3%, Avg loss: 2.167185 

Epoch 2
------------------------
loss: 2.171471 [   64/60000]
loss: 2.163525 [ 6464/60000]
loss: 2.109342 [12864/60000]
loss: 2.121628 [19264/60000]
loss: 2.081834 [25664/60000]
loss: 2.018794 [32064/60000]
loss: 2.053362 [38464/60000]
loss: 1.979922 [44864/60000]
loss: 1.977162 [51264/60000]
loss: 1.901173 [57664/60000]
Test Error: 
 Accuracy: 56.2%, Avg loss: 1.905895 

Epoch 3
------------------------
loss: 1.932469 [   64/60000]
loss: 1.900822 [ 6464/60000]
loss: 1.795963 [12864/60000]
loss: 1.826743 [19264/60000]
loss: 1.721640 [25664/60000]
loss: 1.663977 [32064/60000]
loss: 1.692703 [38464/60000]
loss: 1.602772

In [16]:
# save the model
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


In [17]:
# load the model
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
